# OULAD Mega Table Creator
## Merge All Tables Into One Dataset

This notebook creates a single unified dataset from all 7 OULAD files.

**Why this approach?**
- The raw tables have different granularities (student-level vs click-level)
- We aggregate granular data FIRST, then merge
- Result: One row per student-module-presentation
- Everyone in the group works on the same table!

**Output:** `oulad_mega_table.csv`

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("Libraries loaded!")

Libraries loaded!


In [2]:
# ============================================
# CONFIGURATION - Update this path if needed
# ============================================
DATA_PATH = '.'  # Current directory

print(f"Looking for data files in: {DATA_PATH}")

Looking for data files in: .


## Step 1: Load All Raw Tables

In [3]:
print("Loading all 7 datasets...")
print("-" * 60)

# Load each file
student_info = pd.read_csv(f'{DATA_PATH}/studentInfo.csv')
print(f"✓ studentInfo: {student_info.shape[0]:,} rows x {student_info.shape[1]} cols")

student_vle = pd.read_csv(f'{DATA_PATH}/studentVle.csv')
print(f"✓ studentVle: {student_vle.shape[0]:,} rows x {student_vle.shape[1]} cols")

student_assessment = pd.read_csv(f'{DATA_PATH}/studentAssessment.csv')
print(f"✓ studentAssessment: {student_assessment.shape[0]:,} rows x {student_assessment.shape[1]} cols")

assessments = pd.read_csv(f'{DATA_PATH}/assessments.csv')
print(f"✓ assessments: {assessments.shape[0]:,} rows x {assessments.shape[1]} cols")

vle = pd.read_csv(f'{DATA_PATH}/vle.csv')
print(f"✓ vle: {vle.shape[0]:,} rows x {vle.shape[1]} cols")

courses = pd.read_csv(f'{DATA_PATH}/courses.csv')
print(f"✓ courses: {courses.shape[0]:,} rows x {courses.shape[1]} cols")

student_registration = pd.read_csv(f'{DATA_PATH}/studentRegistration.csv')
print(f"✓ studentRegistration: {student_registration.shape[0]:,} rows x {student_registration.shape[1]} cols")

print("-" * 60)
print("All files loaded successfully!")

Loading all 7 datasets...
------------------------------------------------------------
✓ studentInfo: 32,593 rows x 12 cols
✓ studentVle: 10,655,280 rows x 6 cols
✓ studentAssessment: 173,912 rows x 5 cols
✓ assessments: 206 rows x 6 cols
✓ vle: 6,364 rows x 6 cols
✓ courses: 22 rows x 3 cols
✓ studentRegistration: 32,593 rows x 5 cols
------------------------------------------------------------
All files loaded successfully!


## Step 2: Create Base Table (studentInfo + courses + registration)

In [4]:
print("Creating base table...")
print("-" * 60)

# Start with studentInfo as base
mega_table = student_info.copy()

# Create unique key for each student-module-presentation combination
mega_table['student_module_key'] = (
    mega_table['id_student'].astype(str) + '_' + 
    mega_table['code_module'] + '_' + 
    mega_table['code_presentation']
)

print(f"Base table: {mega_table.shape[0]:,} rows")

# Merge with courses
mega_table = mega_table.merge(
    courses, 
    on=['code_module', 'code_presentation'], 
    how='left'
)
print(f"After merging courses: {mega_table.shape[0]:,} rows, {mega_table.shape[1]} cols")

# Merge with registration
mega_table = mega_table.merge(
    student_registration, 
    on=['code_module', 'code_presentation', 'id_student'], 
    how='left'
)
print(f"After merging registration: {mega_table.shape[0]:,} rows, {mega_table.shape[1]} cols")
print("-" * 60)

Creating base table...
------------------------------------------------------------
Base table: 32,593 rows
After merging courses: 32,593 rows, 14 cols
After merging registration: 32,593 rows, 16 cols
------------------------------------------------------------


## Step 3: Aggregate Assessment Data (student-level)

In [5]:
print("Aggregating assessment data...")
print("-" * 60)

# First, merge student_assessment with assessments metadata
assess_full = student_assessment.merge(assessments, on='id_assessment', how='left')

# Create student-module key
assess_full['student_module_key'] = (
    assess_full['id_student'].astype(str) + '_' + 
    assess_full['code_module'] + '_' + 
    assess_full['code_presentation']
)

# Calculate days early (positive = submitted early)
assess_full['days_early'] = assess_full['date'] - assess_full['date_submitted']

print(f"Assessment records with metadata: {assess_full.shape[0]:,}")

Aggregating assessment data...
------------------------------------------------------------
Assessment records with metadata: 173,912


In [6]:
# Aggregate to student-module level
assess_agg = assess_full.groupby('student_module_key').agg({
    'score': ['mean', 'std', 'min', 'max', 'count'],
    'date_submitted': ['mean', 'min', 'max'],
    'is_banked': 'sum',
    'days_early': ['mean', 'min'],
    'weight': 'sum'
}).reset_index()

# Flatten column names
assess_agg.columns = [
    'student_module_key',
    'assess_score_mean', 'assess_score_std', 'assess_score_min', 'assess_score_max', 'assess_count',
    'assess_submit_day_mean', 'assess_submit_day_min', 'assess_submit_day_max',
    'assess_banked_count',
    'assess_days_early_mean', 'assess_days_early_worst',
    'assess_total_weight'
]

print(f"Assessment aggregated: {assess_agg.shape[0]:,} student-modules")
print(f"Columns: {list(assess_agg.columns[1:])}")
print("-" * 60)

Assessment aggregated: 25,843 student-modules
Columns: ['assess_score_mean', 'assess_score_std', 'assess_score_min', 'assess_score_max', 'assess_count', 'assess_submit_day_mean', 'assess_submit_day_min', 'assess_submit_day_max', 'assess_banked_count', 'assess_days_early_mean', 'assess_days_early_worst', 'assess_total_weight']
------------------------------------------------------------


In [7]:
# Assessment type specific aggregations (TMA, CMA, Exam)
print("Creating assessment type features...")

for atype in ['TMA', 'CMA', 'Exam']:
    type_data = assess_full[assess_full['assessment_type'] == atype]
    type_agg = type_data.groupby('student_module_key').agg({
        'score': ['mean', 'count']
    }).reset_index()
    type_agg.columns = ['student_module_key', f'{atype.lower()}_score_mean', f'{atype.lower()}_count']
    assess_agg = assess_agg.merge(type_agg, on='student_module_key', how='left')
    print(f"  ✓ {atype} features added")

print("-" * 60)

Creating assessment type features...
  ✓ TMA features added
  ✓ CMA features added
  ✓ Exam features added
------------------------------------------------------------


## Step 4: Aggregate VLE Data (student-level)

In [8]:
print("Aggregating VLE data (this may take a moment)...")
print("-" * 60)

# Create student-module key in VLE data
student_vle['student_module_key'] = (
    student_vle['id_student'].astype(str) + '_' + 
    student_vle['code_module'] + '_' + 
    student_vle['code_presentation']
)

print(f"VLE records: {student_vle.shape[0]:,}")

Aggregating VLE data (this may take a moment)...
------------------------------------------------------------
VLE records: 10,655,280


In [9]:
# Basic VLE aggregations
vle_agg = student_vle.groupby('student_module_key').agg({
    'sum_click': ['sum', 'mean', 'std', 'max'],
    'date': ['min', 'max', 'nunique'],
    'id_site': 'nunique'
}).reset_index()

vle_agg.columns = [
    'student_module_key',
    'vle_total_clicks', 'vle_avg_daily_clicks', 'vle_click_std', 'vle_max_daily_clicks',
    'vle_first_access_day', 'vle_last_access_day', 'vle_active_days',
    'vle_unique_resources'
]

# Derived features
vle_agg['vle_engagement_span'] = vle_agg['vle_last_access_day'] - vle_agg['vle_first_access_day']
vle_agg['vle_clicks_per_active_day'] = vle_agg['vle_total_clicks'] / vle_agg['vle_active_days'].replace(0, 1)

print(f"VLE aggregated: {vle_agg.shape[0]:,} student-modules")
print("-" * 60)

VLE aggregated: 29,228 student-modules
------------------------------------------------------------


In [10]:
# Early engagement (first 2 weeks: days 0-14)
print("Creating early engagement features...")

early_vle = student_vle[student_vle['date'] <= 14]
early_agg = early_vle.groupby('student_module_key').agg({
    'sum_click': 'sum',
    'date': 'nunique',
    'id_site': 'nunique'
}).reset_index()
early_agg.columns = ['student_module_key', 'vle_early_clicks', 'vle_early_active_days', 'vle_early_resources']

vle_agg = vle_agg.merge(early_agg, on='student_module_key', how='left')
print("  ✓ Early engagement (days 0-14) features added")

# Pre-course engagement (before day 0)
pre_vle = student_vle[student_vle['date'] < 0]
pre_agg = pre_vle.groupby('student_module_key')['sum_click'].sum().reset_index()
pre_agg.columns = ['student_module_key', 'vle_pre_course_clicks']

vle_agg = vle_agg.merge(pre_agg, on='student_module_key', how='left')
print("  ✓ Pre-course engagement features added")
print("-" * 60)

Creating early engagement features...
  ✓ Early engagement (days 0-14) features added
  ✓ Pre-course engagement features added
------------------------------------------------------------


In [11]:
# Activity type clicks
print("Creating activity type features...")

# Merge VLE with activity types
vle_with_type = student_vle.merge(vle[['id_site', 'activity_type']], on='id_site', how='left')

# Pivot to get clicks by activity type
activity_clicks = vle_with_type.groupby(['student_module_key', 'activity_type'])['sum_click'].sum().unstack(fill_value=0)
activity_clicks = activity_clicks.add_prefix('vle_clicks_').reset_index()

vle_agg = vle_agg.merge(activity_clicks, on='student_module_key', how='left')
print(f"  ✓ Added {activity_clicks.shape[1] - 1} activity type columns")
print("-" * 60)

Creating activity type features...
  ✓ Added 20 activity type columns
------------------------------------------------------------


## Step 5: Merge All Aggregations into Mega Table

In [12]:
print("Merging all aggregations into mega table...")
print("-" * 60)

# Merge assessment aggregations
mega_table = mega_table.merge(assess_agg, on='student_module_key', how='left')
print(f"After assessment merge: {mega_table.shape[1]} columns")

# Merge VLE aggregations
mega_table = mega_table.merge(vle_agg, on='student_module_key', how='left')
print(f"After VLE merge: {mega_table.shape[1]} columns")

print("-" * 60)
print(f"\n✓ MEGA TABLE COMPLETE!")
print(f"  Shape: {mega_table.shape[0]:,} rows x {mega_table.shape[1]} columns")

Merging all aggregations into mega table...
------------------------------------------------------------
After assessment merge: 34 columns
After VLE merge: 68 columns
------------------------------------------------------------

✓ MEGA TABLE COMPLETE!
  Shape: 32,593 rows x 68 columns


## Step 6: Review the Mega Table

In [13]:
print("Mega Table Column Summary:")
print("=" * 60)

# Group columns by source
base_cols = ['code_module', 'code_presentation', 'id_student', 'student_module_key',
             'gender', 'region', 'highest_education', 'imd_band', 'age_band',
             'num_of_prev_attempts', 'studied_credits', 'disability', 'final_result',
             'module_presentation_length', 'date_registration', 'date_unregistration']

assess_cols = [c for c in mega_table.columns if c.startswith('assess_') or c.endswith('_score_mean') or c.endswith('_count')]
vle_cols = [c for c in mega_table.columns if c.startswith('vle_')]

print(f"\n📋 Base columns (studentInfo + courses + registration): {len(base_cols)}")
print(f"📊 Assessment columns: {len(assess_cols)}")
print(f"🖱️ VLE engagement columns: {len(vle_cols)}")
print(f"\nTotal columns: {mega_table.shape[1]}")

Mega Table Column Summary:

📋 Base columns (studentInfo + courses + registration): 16
📊 Assessment columns: 18
🖱️ VLE engagement columns: 34

Total columns: 68


In [14]:
# List all columns
print("\nAll columns in mega table:")
print("-" * 60)
for i, col in enumerate(mega_table.columns, 1):
    print(f"{i:2}. {col}")


All columns in mega table:
------------------------------------------------------------
 1. code_module
 2. code_presentation
 3. id_student
 4. gender
 5. region
 6. highest_education
 7. imd_band
 8. age_band
 9. num_of_prev_attempts
10. studied_credits
11. disability
12. final_result
13. student_module_key
14. module_presentation_length
15. date_registration
16. date_unregistration
17. assess_score_mean
18. assess_score_std
19. assess_score_min
20. assess_score_max
21. assess_count
22. assess_submit_day_mean
23. assess_submit_day_min
24. assess_submit_day_max
25. assess_banked_count
26. assess_days_early_mean
27. assess_days_early_worst
28. assess_total_weight
29. tma_score_mean
30. tma_count
31. cma_score_mean
32. cma_count
33. exam_score_mean
34. exam_count
35. vle_total_clicks
36. vle_avg_daily_clicks
37. vle_click_std
38. vle_max_daily_clicks
39. vle_first_access_day
40. vle_last_access_day
41. vle_active_days
42. vle_unique_resources
43. vle_engagement_span
44. vle_clicks_per_

In [15]:
# Preview
print("\nFirst 5 rows:")
mega_table.head()


First 5 rows:


,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,...,vle_clicks_ouelluminate,vle_clicks_ouwiki,vle_clicks_page,vle_clicks_questionnaire,vle_clicks_quiz,vle_clicks_repeatactivity,vle_clicks_resource,vle_clicks_sharedsubpage,vle_clicks_subpage,vle_clicks_url
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,...,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,32.0,5.0
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,...,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,87.0,48.0
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,22.0,4.0
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,...,0.0,0.0,0.0,0.0,0.0,0.0,19.0,0.0,144.0,90.0
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,...,0.0,0.0,0.0,0.0,0.0,0.0,45.0,0.0,79.0,14.0


In [16]:
# Check missing values
print("\nMissing values per column:")
print("-" * 60)
missing = mega_table.isna().sum()
missing_pct = (missing / len(mega_table) * 100).round(1)

missing_df = pd.DataFrame({'Missing': missing, 'Percent': missing_pct})
missing_df = missing_df[missing_df['Missing'] > 0].sort_values('Missing', ascending=False)

if len(missing_df) > 0:
    print(missing_df)
else:
    print("No missing values!")
    
print("-" * 60)
print("\n⚠️ Note: Missing VLE/assessment values = students with no activity")
print("   These will need to be handled during Data Preparation (fill with 0 or impute)")


Missing values per column:
------------------------------------------------------------
                           Missing  Percent
exam_score_mean              27634     84.8
exam_count                   27634     84.8
date_unregistration          22521     69.1
cma_count                    17493     53.7
cma_score_mean               17493     53.7
assess_score_std              9292     28.5
vle_pre_course_clicks         8784     27.0
tma_score_mean                7805     23.9
tma_count                     7773     23.8
assess_score_max              6773     20.8
assess_score_mean             6773     20.8
assess_score_min              6773     20.8
assess_days_early_worst       6751     20.7
assess_days_early_mean        6751     20.7
assess_total_weight           6750     20.7
assess_submit_day_mean        6750     20.7
assess_count                  6750     20.7
assess_submit_day_min         6750     20.7
assess_submit_day_max         6750     20.7
assess_banked_count           6

In [17]:
# Data types
print("\nData types:")
print("-" * 60)
print(mega_table.dtypes.value_counts())


Data types:
------------------------------------------------------------
float64    54
object     10
int64       4
Name: count, dtype: int64


## Step 7: Save Mega Table

In [18]:
# Save to CSV
output_file = 'oulad_mega_table.csv'
mega_table.to_csv(output_file, index=False)

print("=" * 60)
print("✓ MEGA TABLE SAVED!")
print("=" * 60)
print(f"\nFile: {output_file}")
print(f"Size: {mega_table.shape[0]:,} rows x {mega_table.shape[1]} columns")
print(f"\nShare this file with your group!")
print("Everyone can now work on the same dataset.")

✓ MEGA TABLE SAVED!

File: oulad_mega_table.csv
Size: 32,593 rows x 68 columns

Share this file with your group!
Everyone can now work on the same dataset.


---

## What's Next?

Now that you have `oulad_mega_table.csv`, your group can:

1. **Data Understanding** - Explore distributions, correlations, visualizations
2. **Data Preparation** - Handle missing values, encode categoricals, scale features
3. **Modelling** - Build predictive and clustering models

### Column Reference:

| Prefix | Source | Description |
|--------|--------|-------------|
| (none) | studentInfo | Demographics, final_result (TARGET) |
| (none) | courses | module_presentation_length |
| (none) | registration | date_registration, date_unregistration |
| `assess_` | studentAssessment | Score stats, submission timing |
| `tma_`, `cma_`, `exam_` | studentAssessment | Type-specific scores |
| `vle_` | studentVle | Click counts, engagement metrics |
| `vle_clicks_` | studentVle + vle | Clicks by activity type |
| `vle_early_` | studentVle | First 2 weeks engagement |
| `vle_pre_course_` | studentVle | Before day 0 engagement |